In [1]:
import torch
import pandas as pd
from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments
import sentencepiece as spm
from datasets import load_dataset
from accelerate import Accelerator
import numpy as np
from sklearn.model_selection import train_test_split


2024-08-12 11:05:43.145114: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-12 11:05:43.145243: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-12 11:05:43.280376: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
import os
import pandas as pd
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, Trainer, TrainingArguments

# Paths to the model files
model_path = "/kaggle/input/cg/transformers/default/1"

# Load the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(model_path,use_fast=False)
model = AutoModelForSeq2SeqLM.from_pretrained(model_path)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [5]:
from sklearn.model_selection import train_test_split

def load_and_split_data(file_path, test_size=0.2):
    df = pd.read_excel(file_path)
    train_data, valid_data = train_test_split(df, test_size=test_size, random_state=42)
    return train_data, valid_data

# Load and split data
train_data, valid_data = load_and_split_data('/kaggle/input/tcgui-acp/ACPgui.XLSX')

In [38]:
import pandas as pd

# Load your data
df = pd.read_excel('/kaggle/input/gui-acp-tc/ACPgui.XLSX')

# Compute text lengths
df['action_length'] = df['Actions'].apply(lambda x: len(str(x).split()))
df['code_length'] = df['Code Groovy'].apply(lambda x: len(str(x).split()))

# Calculate statistics
print("Action Lengths: ", df['action_length'].describe())
print("Code Lengths: ", df['code_length'].describe())

Action Lengths:  count    1208.000000
mean        7.097682
std         4.376490
min         1.000000
25%         4.000000
50%         6.000000
75%         8.250000
max        40.000000
Name: action_length, dtype: float64
Code Lengths:  count    1208.000000
mean        4.062086
std         4.632326
min         1.000000
25%         1.000000
50%         3.000000
75%         5.000000
max        60.000000
Name: code_length, dtype: float64


In [39]:
max_action_length = df['action_length'].quantile(0.95)
max_code_length = df['code_length'].quantile(0.95)

print("Optimal Max Length for Actions:", max_action_length)
print("Optimal Max Length for Code:", max_code_length)

Optimal Max Length for Actions: 15.0
Optimal Max Length for Code: 10.0


In [6]:
def save_data_to_text(data, file_path):
    with open(file_path, 'w', encoding='utf-8') as f:
        for index, row in data.iterrows():
            action = str(row['Actions']) if not pd.isna(row['Actions']) else ""
            code = str(row['Code Groovy']) if not pd.isna(row['Code Groovy']) else ""
            f.write(action + '\t' + code + '\n')


# Save train and validation data to text files
save_data_to_text(train_data, 'train.txt')
save_data_to_text(valid_data, 'valid.txt')

In [7]:
# Define function to tokenize a single sample
def tokenize_sample(sample, tokenizer, max_length):
    action_text = str(sample['Actions'])
    code_text = str(sample['Code Groovy'])

    input_encodings = tokenizer(
        action_text,
        truncation=True,
        padding='max_length',
        max_length=max_length
    )

    target_encodings = tokenizer(
        code_text,
        truncation=True,
        padding='max_length',
        max_length=max_length
    )

    return {
        'input_ids': torch.tensor(input_encodings['input_ids']),
        'attention_mask': torch.tensor(input_encodings['attention_mask']),
        'labels': torch.tensor(target_encodings['input_ids'])
    }


In [10]:
# Step 4: Tokenize the Data
#def tokenize_data(file_path, sp_model):
    with open(file_path, 'r', encoding='utf-8') as f:
        lines = f.readlines()
    tokenized_data = []
    for line in lines:
        line = line.strip()
        if '\t' in line:
            action, code = line.split('\t', 1)
            tokenized_action = sp_model.encode_as_pieces(action)
            tokenized_code = sp_model.encode_as_pieces(code)
            tokenized_data.append((tokenized_action, tokenized_code))
        else:
            print(f"Skipping line: {line}")
    return tokenized_data

In [11]:
# Tokenize the training and validation data
#tokenized_train_data = tokenize_data('train.txt', sp)
#tokenized_valid_data = tokenize_data('valid.txt', sp)


Skipping line: 
Skipping line: 
Skipping line: WebUI.maximizeWindow()
Skipping line: CustomKeywords.'gui.axefinance.functionality.EditOrDelete_FieldsetOrGrid.execute'('New Display', 'Delete', '')
Skipping line: 
Skipping line: CustomKeywords.'gui.axefinance.keyword.click.ClickButtonByName.click'('Yes')
Skipping line: 
Skipping line: CustomKeywords.'gui.axefinance.functionality.CheckDeleteAction.check'()
Skipping line: }
Skipping line: WebUI.verifyElementAttributeValue(findTestObject('GUI/Xpath', [('xpath') : '//input[@name="a__transaction__transaction_shortname"]']),
Skipping line: 'aria-readonly', 'false', 0)
Skipping line: 
Skipping line: 'Check fields not read only'
Skipping line: WebUI.verifyElementAttributeValue(findTestObject('GUI/Xpath', [('xpath') : '//input[@name="h__transaction__transaction_type_id"]']),
Skipping line: 'aria-readonly', 'false', 0)
Skipping line: 
Skipping line: 'Check fields not read only'
Skipping line: WebUI.verifyElementAttributeValue(findTestObject('GUI/X

In [8]:
def compute_max_length(tokenized_data):
    return max(len(action) + len(code) for action, code in tokenized_data)

max_seq_length = compute_max_length(tokenized_train_data)
max_seq_length_valid = compute_max_length(tokenized_valid_data)
max_seq_length = max(max_seq_length, max_seq_length_valid)

NameError: name 'tokenized_train_data' is not defined

In [9]:
# Prepare datasets
# Step 3: Define custom dataset class
# Define dataset class
# Define custom dataset class
class CodeDataset(torch.utils.data.Dataset):
    def __init__(self, data, tokenizer, max_length):
        self.data = data
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        sample = self.data.iloc[idx]
        tokenized_sample = tokenize_sample(sample, self.tokenizer, self.max_length)
        return tokenized_sample



In [10]:
train_dataset = CodeDataset(train_data, tokenizer, max_length=300)
valid_dataset = CodeDataset(valid_data, tokenizer, max_length=300)





In [11]:
len(train_dataset)

1321

In [12]:
# Define training arguments with optimizations
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs= 10,  # Increased number of epochs
    per_device_train_batch_size=1,  # Larger batch size
    per_device_eval_batch_size=1,
    warmup_steps=1000,  # Adjust warmup steps
    weight_decay=0.01,
    logging_dir='./logs',
    fp16=True,  # Enable mixed precision training
    save_steps=500,
    eval_strategy="steps",
    save_total_limit=3,  # Keep only the last 3 checkpoints
)

In [13]:
accelerator = Accelerator()

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,)

In [14]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Step,Training Loss,Validation Loss
500,0.538400,0.184416
1000,0.164200,0.120982
1500,0.113300,0.096465
2000,0.107300,0.081096
2500,0.070900,0.073718
3000,0.059900,0.069982
3500,0.058800,0.064212
4000,0.060100,0.059898
4500,0.058900,0.058837
5000,0.037300,0.056161


TrainOutput(global_step=13210, training_loss=0.06507387439199327, metrics={'train_runtime': 2874.8746, 'train_samples_per_second': 4.595, 'train_steps_per_second': 4.595, 'total_flos': 4713476797440000.0, 'train_loss': 0.06507387439199327, 'epoch': 10.0})

In [15]:
trainer.train()

Step,Training Loss,Validation Loss
500,0.025500,0.042507
1000,0.019800,0.044208
1500,0.020300,0.041237
2000,0.028500,0.038680
2500,0.019700,0.038062
3000,0.018500,0.037098
3500,0.019300,0.037194
4000,0.027200,0.037286
4500,0.029800,0.037407
5000,0.020200,0.037489


TrainOutput(global_step=13210, training_loss=0.0221995624784806, metrics={'train_runtime': 2876.0137, 'train_samples_per_second': 4.593, 'train_steps_per_second': 4.593, 'total_flos': 4713476797440000.0, 'train_loss': 0.0221995624784806, 'epoch': 10.0})

In [17]:
# Save the fine-tuned model
model.save_pretrained('/kaggle/working/fine_tuned_modelT5')
tokenizer.save_pretrained('/kaggle/working/fine_tuned_modelT5')

('/kaggle/working/fine_tuned_modelT5/tokenizer_config.json',
 '/kaggle/working/fine_tuned_modelT5/special_tokens_map.json',
 '/kaggle/working/fine_tuned_modelT5/spiece.model',
 '/kaggle/working/fine_tuned_modelT5/added_tokens.json')

In [18]:
!zip -r fine_tuned_modelT5.zip ./fine_tuned_modelT5


/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


  adding: fine_tuned_modelT5/ (stored 0%)
  adding: fine_tuned_modelT5/spiece.model (deflated 48%)
  adding: fine_tuned_modelT5/model.safetensors (deflated 8%)
  adding: fine_tuned_modelT5/config.json (deflated 62%)
  adding: fine_tuned_modelT5/special_tokens_map.json (deflated 85%)
  adding: fine_tuned_modelT5/tokenizer_config.json (deflated 94%)
  adding: fine_tuned_modelT5/added_tokens.json (deflated 83%)
  adding: fine_tuned_modelT5/generation_config.json (deflated 29%)


In [27]:
model_path = '/kaggle/working/fine_tuned_modelT5'
tokenizer = AutoTokenizer.from_pretrained(model_path, use_fast=False)
model = AutoModelForSeq2SeqLM.from_pretrained(model_path)

# Set the model to evaluation mode
model.eval()
max_length=500
# Define a function for generating code from actions
def generate_code(actions, max_new_tokens=200):
    inputs = tokenizer(actions, return_tensors='pt', padding=True, truncation=True)
    with torch.no_grad():
        outputs = model.generate(**inputs, max_new_tokens=max_new_tokens)
    decoded_outputs = [tokenizer.decode(output, skip_special_tokens=True) for output in outputs]
    return decoded_outputs


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [28]:
# Test the model with some sample actions
sample_actions = [
    "Connect to GUI","Pin sidebar"

]

generated_code = generate_code(sample_actions)
for action, code in zip(sample_actions, generated_code):
    print(f"Action: {action}")
    print(f"Generated Code: {code}\n")

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Action: Connect to GUI
Generated Code: WebUI.callTestCase(findTestCase('Login/Login GUI'), [:], FailureHandling.STOP_ON_FAILURE)

Action: Pin sidebar
Generated Code: CustomKeywords.'gui.axefinance.keyword.click.PinSideBar.pin'()

